In [ ]:
import os
from lxml import etree as ET

In [ ]:
# Getting all usernames (E.g., faculty usernames)

usersEtree = ET.parse('output/users.xml')
users = usersEtree.findall('.//User')

In [ ]:
# Check that no two users have the same 'username'

# found_users = [i.attrib['username'] for i in users]
# found_users_set = set(found_users)
# print(len(found_users) - len(found_users_set))  # all unique names?
# print(found_users[0])  # display an example
# print(len(users))  # display how many there are

#### Take aways

2208 total 'User'

All 'User' items have a unique 'username' attribute

In [ ]:
# Check for error xmls

# userfiles = sorted(os.listdir('output/users/'))
# count = 0

# for file in userfiles:
#     eTree = ET.parse(f"output/users/{file}")
#     if "error" in eTree.getroot().tag.lower():
#         print(os.path.splitext(file)[0])
#         count += 1
# #         with open(f"output/users/{file}") as f:
# #             print(f.read())
# print(f"errored Users count: {count}")

#### Take aways

34 Users gave an error when doing 'INDIVIDUAL-ACTIVITIES-University'.

These are:  "weisa", "braunr", "mcb4548", "carabellin", "chapmanc", "cohens", "culpa", "DIEPPAS", "gorae", "hoadleyd", "jrh4152", "jarrellb", "keepw", "knoxj", "lambertonc", "lawsonw", "learw", "mml7328", "luopaj", "macleodw", "smm9756", "mcintoshb", "nelsoncl", "powella", "fac_reports", "richardsa", "sahljs", "databackup_service", "crs5798", "stuartal", "web_services_vivo", "westgatea", "wilsonca"

An example error response:

```
<?xml version="1.0" encoding="UTF-8"?>
<Error>The following errors were detected:
	<Message>Invalid QueryParameter Values specified: 404/USERNAME/OWNER/[westgatea]</Message>
</Error>
```

Using Google to investigate whether the ERROR response was justifiable.  Some individuals are intended for exclusion from vivo -- including non-human users and some others.  Below, categorizing the ERROR users:

The affected Users are:

3 non-human users: {'databackup_service', 'fac_reports', and 'web_services_vivo'}

4 administrative assistants: {'DIEPPAS', 'carabellin', 'culpa', 'sahljs'}

3 IT: {'chapmanc', 'luopaj', 'powella'}

2 IRP: {'cohens', 'lawsonw'}

1 Dean's Office: {'wilsonca'}

1 possibly name changed: {'gorea' may now be 'williamsone'}

19 unknown reason: {'braunr', 'crs5798', 'gorae', 'hoadleyd', 'jarrellb', 'jrh4152', 'keepw', 'knoxj', 'lambertonc', 'learw', 'macleodw', 'mcb4548', 'nelsoncl', 'richardsa', 'smm9756', 'stuartal', 'weisa', 'wellswd', 'westgatea'}

Question:  Who do we contact about the possible error related to name change?
    
Question:  How do we confirm the intent is to exclude these users from Vivo?

In [ ]:
# Describing the top level structure of all user files

# userfiles = sorted(os.listdir('output/users'))
# len_Message, len_Record = set(), set()

# all_2nd_tags = set()
# for file in userfiles:
#     etree = ET.parse(f"output/users/{file}")
#     len_Message.add(len(etree.findall('Message')))
#     len_Record.add(len(etree.findall('{http://www.digitalmeasures.com/schema/data}Record')))
#     for i in etree.getroot().iterchildren():
#         all_2nd_tags.add(i.tag)
# print(all_2nd_tags)
# print(len_Message, len_Record)

#### Take aways

Userfiles have a "Data" elem with child "Message" or "Record".

The 34 error responses have a Message.

All the other have a Record.

Never more than one Message or Record.  Never both.

In [ ]:
# Describing the 2nd level structure of all user files

# record_children = set()
# userfiles = sorted(os.listdir('output/users'))

# for file in userfiles:
#     etree = ET.parse(f"output/users/{file}")
#     record_tag = etree.find('{http://www.digitalmeasures.com/schema/data}Record')
#     if not record_tag:
#         continue
#     child_tags = record_tag.getchildren()
#     for i in child_tags:
#         record_children.add(i.tag)

# for i in record_children:
#     print(i)

#### Take aways

Below a Record we find these tags: (which are a subset of the entities.xml response)  There's a chance IRP may add other tags at some later date.

    ACADVISE
    ADMIN
    ADMIN_ASSIGNMENTS
    ADMIN_PERM
    AWARDHONOR
    BIOSKETCH
    CONGRANT
    CONSULT
    CURRICULUM
    DSL
    EDUCATION
    FACDEV
    INTELLCONT
    INTELLPROP
    LICCERT
    MEDCONT
    MEMBER
    NCTEACH
    NIH_BIOSKETCH
    PASTHIST
    PCI
    PERFORM_EXHIBIT
    PRESENT
    RESPROG
    SCHTEACH
    SERVICE_COLLEGE
    SERVICE_DEPARTMENT
    SERVICE_PROFESSIONAL
    SERVICE_PUBLIC
    SERVICE_UNIVERSITY
    TEACHING_QUAL

    {http://www.digitalmeasures.com/schema/data-metadata}IndexEntry

In [ ]:
# Describing the Presentation tag

# nsmap = {'xmlns': "http://www.digitalmeasures.com/schema/data",
#          'dmd': "http://www.digitalmeasures.com/schema/data-metadata"
#         }
# all_presentations = set()

# def grab_all_presentations(file):
#     etree = ET.parse(f"output/users/{file}")
#     #     print([i for i in etree.getroot().iterchildren()])
#     for i in etree.xpath('./xmlns:Record/xmlns:PRESENT', namespaces=nsmap):
#         if i.attrib['id'] in all_presentations:
#             print(file, i.attrib['id'])
#             continue
#         all_presentations.add(i.attrib['id'])

# userfiles = sorted(os.listdir('output/users'))
# for file in userfiles:
#     grab_all_presentations(file)

#### Take aways

A PRESENT, etc can show up in two User files.  

For example, leel & casterellag co-presented id "187389001728" ("IT Audit:  Supporting the Audit Team with the AICPA General Ledger Data Standard").

Assuming, we perform the ETL via User files, that PRESENT could be entered twice in Vivo unless we de-duplicate them before entry.

I believe we can assume equality between all elems with the same 'id' attribute.

In [ ]:
# Check that all users have the same subtags

# expected_subtags = set()

# extra_subtags = set()
# missing_subtags = dict()

# for i in users[0].getchildren():
#     expected_subtags.add(i.tag)
# for user in users:
#     user_subtags = set(i.tag for i in user.getchildren())
#     if user_subtags != expected_subtags:
#         extra = user_subtags - expected_subtags
#         if extra:
#             extra_subtags.add(user)
#         missing = expected_subtags - user_subtags
#         if missing:
#             missing_subtags[user] = missing

# for i in extra_subtags:
#     print(i.attrib)

#### Take aways

3 fake users {'web_services_vivo', 'databackup_service', 'fac_reports'}
These have 'LocalAuthentication' subtag.

'bowengl' and 'padgettm' have a subtag 'LocalAuthentication', while the rest have 'LDAPAuthentication'

Probably a moot point.

In [ ]:
# Collect the elements that have an empty text

# empty_subtags = dict()

# for user in users:
#     blank_subtags = {
#         i.tag
#         for i in user.getchildren()
#         if not i.text
#     }
#     if blank_subtags:
#         empty_subtags[user.attrib['username']] = blank_subtags

# empty_subtags

In [ ]:
# MiddleName was a common element with no text.  LDAPAuthentication was another.
# Count how many records have an empty text for 'MiddleName'

# count = 0
# for k, v in missing_subtags.items():
#     if v == {'MiddleName'}:
#         count += 1
#     else:
#         print(k.attrib, v)
# print(f"{count} users with 'Middlename' attribute")

#### Take aways

491 users have no 'MiddleName' attribute

5 users have no 'LDAPAuthentication' attribute:

    bowengl
    padgettm
    fac_reports
    web_services_vivo
    databackup_service
    
Probably a moot point

In [ ]:
# Collect the other tags with empty text

# print(len([user.attrib['username']
#        for user in users
#        for subtag in user.findall('LDAPAuthentication')
#        if not subtag.text
#       ]))
# print([user.attrib['username']
#        for user in users
#        for subtag in user.findall('LDAPAuthentication')
#        if subtag.text
#       ])
# print(len([user.attrib['username']
#        for user in users
#        for subtag in user.findall('Item')
#        if not subtag.text
#       ]))
# print([user.attrib['username']
#        for user in users
#        for subtag in user.findall('Item')
#        if subtag.text
#       ])

# print([(user, value)
#      for user, value in empty_subtags.items()
#      if value != {'LDAPAuthentication', 'Item'}
#       ])


# for k, v in empty_subtags.items():
#     print(k, v)
#     break

#### Take aways

5 Users have 'LocalAuthentication' with empty text.
All others have 'LDAPAuthentication' with empty text.

All Users have 'Item' with attributes, but empty text.  The Item attribs are always 'type'='simple' and a unique 'href'. 

3 Users have 'Email' with empty text.  {'fac_reports', 'databackup_service', 'web_services_vivo'}

Probably a moot point.

In [ ]:
# Collect info on User element attributes

# user_attribs = {attrib
#                for user in users
#                for attrib in user.attrib}

# no_enabled, no_BannerID = 0, 0

# for user_attrib in user_attribs:
#     print('*******', user_attrib)
#     for user in users:
#         if not user.get(user_attrib):
#             print(user.get('username'))
#             if user_attrib == 'BannerID':
#                 no_BannerID += 1
#             if user_attrib == 'enabled':
#                 no_enabled += 1
# print(no_enabled, no_BannerID)

#### Take aways

Four attribs on User object:  {'BannerID', 'userID', 'enabled', 'username'}

625 Users have an 'enabled'.  All 'enabled' are false.  Probably a moot point.

1443 Users have a 'BannerID'.  Probably a moot point.

All Users have a 'userID' and a 'username'.  This seems relevant, and looks correct.

In [ ]:
# Describing the PERFORM_EXHIBIT tag

nsmap = {'xmlns': "http://www.digitalmeasures.com/schema/data",
         'dmd': "http://www.digitalmeasures.com/schema/data-metadata"
        }

def how_many_instances(file, tag):
    etree = ET.parse(f"../app/output/users/{file}")
    for i in etree.xpath(tag, namespaces=nsmap):
        this_perform_subtags = set()
        for j in i.iterchildren():
            if not all_subtags.get(j.tag):
                all_subtags[j.tag] = 'single'
                this_perform_subtags.add(j.tag)
                continue
            if all_subtags[j.tag] == 'multiple':
                this_perform_subtags.add(j.tag)
                continue
            if all_subtags[j.tag] == 'single' and j.tag in this_perform_subtags:
                all_subtags[j.tag] = 'multiple'
            this_perform_subtags.add(j.tag)

In [ ]:
all_pe = set()
all_subtags = dict()

userfiles = sorted(os.listdir('../app/output/users'))
for file in userfiles:
#     how_many_instances(file, f'./xmlns:Record/xmlns:PERFORM_EXHIBIT')
#     how_many_instances(file, f'./xmlns:Record/xmlns:PERFORM_EXHIBIT/xmlns:PERFORM_EXHIBIT_CONTRIBUTERS')
#     how_many_instances(file, f'./xmlns:Record/xmlns:INTELLPROP')
    how_many_instances(file, f'./xmlns:Record/xmlns:CONGRANT')

sorted(all_subtags.items())

In [ ]:
def what_values_in_key(tag):
    all_values = set()
    for file in sorted(os.listdir('../app/output/users')):
        etree = ET.parse(f"../app/output/users/{file}")
        for i in etree.xpath(tag, namespaces=nsmap):
            all_values.add(i.text)
    return all_values
        

In [ ]:
# what_values_in_key('./xmlns:Record/xmlns:CONGRANT/xmlns:AWARDORG')
# what_values_in_key('./xmlns:Record/xmlns:CONGRANT/xmlns:SPONORG')
# what_values_in_key('./xmlns:Record/xmlns:CONGRANT/xmlns:TYPE')
# what_values_in_key('./xmlns:Record/xmlns:CONGRANT/xmlns:STATUS')
what_values_in_key('./xmlns:Record/xmlns:CONGRANT/xmlns:CLASSIFICATION')